In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words

import string
import re
from collections import Counter
from collections import OrderedDict
import unidecode
import requests
import json
from pprint import pprint

In [45]:
people = set()

In [46]:
url = 'https://query.wikidata.org/sparql'

In [47]:
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
#          wdt:P106/wdt:P279* wd:Q2526255; #uncomment for     FILM director (no award for TV director)
#          wdt:P106/wdt:P279* wd:Q10800557; #uncomment for    FILM actor (don't just use actor)
#          wdt:P106/wdt:P279* wd:Q10798782; #uncomment for    TV actor (don't just use actor)
#          wdt:P106/wdt:P279* wd:Q36834; #uncomment for       composer (cannot use songwriter)
#          wdt:P106/wdt:P279* wd:Q28389; #uncomment for       screenwriter
          wdt:P106/wdt:P279* wd:Q177220; #uncomment for       singer
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [48]:
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
#          wdt:P106/wdt:P279* wd:Q2526255; #uncomment for     FILM director (no award for TV director)
#          wdt:P106/wdt:P279* wd:Q10800557; #uncomment for    FILM actor (don't just use actor)
          wdt:P106/wdt:P279* wd:Q10798782; #uncomment for    TV actor (don't just use actor)
#          wdt:P106/wdt:P279* wd:Q36834; #uncomment for       composer (cannot use songwriter)
#          wdt:P106/wdt:P279* wd:Q28389; #uncomment for       screenwriter
#          wdt:P106/wdt:P279* wd:Q177220; #uncomment for       singer
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [49]:
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
#          wdt:P106/wdt:P279* wd:Q2526255; #uncomment for     FILM director (no award for TV director)
          wdt:P106/wdt:P279* wd:Q10800557; #uncomment for    FILM actor (don't just use actor)
#          wdt:P106/wdt:P279* wd:Q10798782; #uncomment for    TV actor (don't just use actor)
#          wdt:P106/wdt:P279* wd:Q36834; #uncomment for       composer (cannot use songwriter)
#          wdt:P106/wdt:P279* wd:Q28389; #uncomment for       screenwriter
#          wdt:P106/wdt:P279* wd:Q177220; #uncomment for       singer
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [29]:
data = pd.read_json('../gg2015.json')
data = pd.DataFrame(data)

In [31]:
present = ['present', 'annouc', 'introduc']
data = data[data['text'].str.contains('|'.join(present))]

In [32]:
data = data['text'].values.tolist()

In [33]:
#print(data[:10])

In [34]:
f = open('english.txt', 'r')
stop_words = f.read().splitlines()
#print(stopwords)

In [35]:
OFFICIAL_AWARDS_1315 = ['cecil b. demille award', 'best motion picture - drama', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best motion picture - comedy or musical', 'best performance by an actress in a motion picture - comedy or musical', 'best performance by an actor in a motion picture - comedy or musical', 'best animated feature film', 'best foreign language film', 'best performance by an actress in a supporting role in a motion picture', 'best performance by an actor in a supporting role in a motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best television series - comedy or musical', 'best performance by an actress in a television series - comedy or musical', 'best performance by an actor in a television series - comedy or musical', 'best mini-series or motion picture made for television', 'best performance by an actress in a mini-series or motion picture made for television', 'best performance by an actor in a mini-series or motion picture made for television', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']

In [36]:
def parseAward(award):
    award = re.split('\W+', award)
    award = [i for i in award if i not in stop_words]
    award = list(set(award))
    return award

In [37]:
def getCategoriesDict():
    categories_dict = dict()
    for a in OFFICIAL_AWARDS_1315:
        terms = parseAward(a)
        categories_dict[a] = terms

    return categories_dict

In [38]:
categories_dict = getCategoriesDict()

In [39]:
categories_dict

{'cecil b. demille award': ['cecil', 'demille', 'award'],
 'best motion picture - drama': ['picture', 'drama', 'motion'],
 'best performance by an actress in a motion picture - drama': ['motion',
  'drama',
  'actress',
  'picture',
  'performance'],
 'best performance by an actor in a motion picture - drama': ['motion',
  'drama',
  'actor',
  'picture',
  'performance'],
 'best motion picture - comedy or musical': ['comedy',
  'picture',
  'musical',
  'motion'],
 'best performance by an actress in a motion picture - comedy or musical': ['picture',
  'musical',
  'motion',
  'actress',
  'comedy',
  'performance'],
 'best performance by an actor in a motion picture - comedy or musical': ['comedy',
  'musical',
  'motion',
  'actor',
  'picture',
  'performance'],
 'best animated feature film': ['feature', 'film', 'animated'],
 'best foreign language film': ['film', 'foreign', 'language'],
 'best performance by an actress in a supporting role in a motion picture': ['role',
  'supporti

In [40]:
def filter0(data, list1):
    synonyms = {
        'motion' : ['motion picture', 'motion', 'picture', 'movie'],
        'picture' : ['motion picture', 'motion', 'picture', 'movie'],
        'television' : ['television', 'tv'],
        'mini' : ['mini-series', 'mini', 'series', 'miniseries'],
        'series' : ['mini-series', 'mini', 'series', 'miniseries']
    }
    
    result = []
    
    list1 = [i for i in list1 if i != 'performance' and i != 'role']

    for tweet in data:
        cond = True
        for i in list1:
            if i in synonyms:
                if all(j not in tweet.lower() for j in synonyms[i]):
                    cond = False
            elif i not in tweet.lower():
                cond = False
        if cond:
            result.append(tweet)

    return result

In [41]:
def extractPresenters(data, list1):
    #print(data[0])
    result = []
       
    #tt = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=True)
    translator = str.maketrans('', '', string.punctuation)
    remove_terms = ['#goldenglobes', 'golden globes', '#goldenglobe', 'golden globe', 'goldenglobes', 'goldenglobe', 'golden', 'globe', 'globes']
    stop = remove_terms + list1
    
    for tweet in data:
        #print(tweet)
        tweet = re.sub("\d+", "", tweet) #strip nums
        tweet = re.sub(r'http\S+', '', tweet) #strip urls
        tweet = re.sub(r'#\S+', '', tweet) #strip hashtags
        tweet = tweet.translate(translator) #strip non-alphanumeric characters
        tweet = tweet.split() #tokenize
        #tweet = [term for term in tweet if term.lower() not in stop_words] #remove stop words
        for i in stop:
            for j in tweet:
                if i.lower() in j.lower():
                    tweet.remove(j)
        result.append(tweet)
        
    #print(result[:20])
        
    grams = [];

    for tweet in result:
        if tweet:
            gram = list(nltk.everygrams(tweet, 2, 3))
            #print(bigram[:10])
            for g in gram:
                if len(g) == 2:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])):
                        grams.append(g)
                else:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[2])):
                        grams.append(g)

  
    fdist = nltk.FreqDist(grams)
    #print(fdist)

    try:
        names = fdist.most_common()
        #names = [' '.join(i[0]) for i in fdist.most_common()]
    except:
        names = "nothing here"
    
    return names

In [42]:
def getPresenters():
    d = {}
    for award in OFFICIAL_AWARDS_1315:
        d["{0}".format(award)] = filter0(data, categories_dict[award])
    
    presenters = {}
    for award in OFFICIAL_AWARDS_1315:
        presenters["{0}".format(award)] = extractPresenters(d[award], categories_dict[award])
    
    return presenters

In [43]:
getPresenters()

{'cecil b. demille award': [(('George', 'Clooney'), 40),
  (('Don', 'Cheadle'), 28),
  (('Julianna', 'Margulies'), 27),
  (('Julianna', 'Marguiles'), 5),
  (('Carol', 'Hathaway'), 2),
  (('Doug', 'Ross'), 2),
  (('Lifetime', 'Achievement'), 2),
  (('George', 'Clooneys'), 2),
  (('Robert', 'Downey'), 1),
  (('Downey', 'Jr'), 1),
  (('Jodie', 'Foster'), 1),
  (('Robert', 'Downey', 'Jr'), 1),
  (('So', 'Carol'), 1),
  (('So', 'Carol', 'Hathaway'), 1),
  (('Clooney', 'How'), 1),
  (('George', 'Clooney', 'How'), 1),
  (('Julianne', 'Margulies'), 1),
  (('Clooney', 'So'), 1),
  (('George', 'Clooney', 'So'), 1),
  (('Somewhere', 'Richard'), 1),
  (('Richard', 'Kind'), 1),
  (('Somewhere', 'Richard', 'Kind'), 1),
  (('Margulies', 'Don'), 1),
  (('Julianna', 'Margulies', 'Don'), 1),
  (('Margulies', 'Don', 'Cheadle'), 1),
  (('Alicia', 'Florrick'), 1),
  (('Col', 'Rhodes'), 1),
  (('Achievement', 'George'), 1),
  (('Lifetime', 'Achievement', 'George'), 1),
  (('Achievement', 'George', 'Clooney'

In [44]:
for tweet in data:
    if 'megan fox' in tweet.lower():
        print(tweet)